In [14]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import operator
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from numpy import unique
from numpy import reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Conv2D, Dense, BatchNormalization, Flatten, MaxPooling1D, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import keras
from sklearn.metrics import classification_report

ModuleNotFoundError: No module named 'keras.layers.core'

In [2]:
weather_data_path = 'C:/Users/kuohe/ClimateWins/02 Data/Dataset-weather-prediction-dataset-processed (2).csv'
pleasant_data_path = 'C:/Users/kuohe/ClimateWins/02 Data/pleasant weather.csv'

In [3]:
weather_data = pd.read_csv(weather_data_path)
pleasant_data = pd.read_csv(pleasant_data_path)

In [4]:
print(weather_data.head())
print(pleasant_data.head())

       DATE  MONTH  BASEL_cloud_cover  BASEL_wind_speed  BASEL_humidity  \
0  19600101      1                  7               2.1            0.85   
1  19600102      1                  6               2.1            0.84   
2  19600103      1                  8               2.1            0.90   
3  19600104      1                  3               2.1            0.92   
4  19600105      1                  6               2.1            0.95   

   BASEL_pressure  BASEL_global_radiation  BASEL_precipitation  \
0           1.018                    0.32                 0.09   
1           1.018                    0.36                 1.05   
2           1.018                    0.18                 0.30   
3           1.018                    0.58                 0.00   
4           1.018                    0.65                 0.14   

   BASEL_snow_depth  BASEL_sunshine  ...  VALENTIA_cloud_cover  \
0                 0             0.7  ...                     5   
1                 0 

I would choose RNN because LSTM units in recurrent neural networks (RNNs) can capture long-term dependencies and handle sequences well, they are perfect for weather data analysis. They are especially useful for long-term weather pattern prediction because they are resilient to gaps and irregularities in the data.


In [5]:
# def build_rnn_model(input_shape):
#     model = Sequential()
    
#     model.add(LSTM(units=50, return_sequences=True, input_shape=input_shape))
#     model.add(Dropout(0.2))  
    
#     model.add(LSTM(units=50))
#     model.add(Dropout(0.2))
    
#     model.add(Dense(1, activation='sigmoid'))
    
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
#     return model


In [6]:
weather_data.drop(columns=['DATE', 'MONTH'], inplace=True)

pleasant_data.drop(columns=['DATE'], inplace=True)

In [7]:
print(weather_data.head())

   BASEL_cloud_cover  BASEL_wind_speed  BASEL_humidity  BASEL_pressure  \
0                  7               2.1            0.85           1.018   
1                  6               2.1            0.84           1.018   
2                  8               2.1            0.90           1.018   
3                  3               2.1            0.92           1.018   
4                  6               2.1            0.95           1.018   

   BASEL_global_radiation  BASEL_precipitation  BASEL_snow_depth  \
0                    0.32                 0.09                 0   
1                    0.36                 1.05                 0   
2                    0.18                 0.30                 0   
3                    0.58                 0.00                 0   
4                    0.65                 0.14                 0   

   BASEL_sunshine  BASEL_temp_mean  BASEL_temp_min  ...  VALENTIA_cloud_cover  \
0             0.7              6.5             0.8  ...          

In [8]:
print(pleasant_data.head())

   BASEL_pleasant_weather  BELGRADE_pleasant_weather  \
0                       0                          0   
1                       0                          0   
2                       0                          0   
3                       0                          0   
4                       0                          0   

   BUDAPEST_pleasant_weather  DEBILT_pleasant_weather  \
0                          0                        0   
1                          0                        0   
2                          0                        0   
3                          0                        0   
4                          0                        0   

   DUSSELDORF_pleasant_weather  HEATHROW_pleasant_weather  \
0                            0                          0   
1                            0                          0   
2                            0                          0   
3                            0                          0   
4              

In [9]:
weather_data.isnull().sum().sum()

0

In [10]:
pleasant_columns = list(pleasant_data.columns)
station_names = [x.split("_")[0] for x in pleasant_columns]

In [11]:
len(station_names)

15

In [12]:
cols_to_keep = []
for col in station_names:
    for weather_col in weather_data.columns:
        if col in weather_col:
            cols_to_keep.append(weather_col)

In [13]:
len(cols_to_keep)

147

In [14]:
basel_cols = [col for col in cols_to_keep if 'BASEL' in col]
belgrade_cols = [col for col in cols_to_keep if 'BELGRADE' in col]
stockholm_cols = [col for col in cols_to_keep if 'STOCKHOLM' in col]

In [15]:
basel_cols

['BASEL_cloud_cover',
 'BASEL_wind_speed',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_snow_depth',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max']

In [16]:
belgrade_cols

['BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max']

In [17]:
stockholm_cols

['STOCKHOLM_cloud_cover',
 'STOCKHOLM_pressure',
 'STOCKHOLM_global_radiation',
 'STOCKHOLM_precipitation',
 'STOCKHOLM_sunshine',
 'STOCKHOLM_temp_mean',
 'STOCKHOLM_temp_min',
 'STOCKHOLM_temp_max']

In [18]:
weather_attributes=[col.replace('BELGRADE_', '') for col in belgrade_cols]

In [19]:
weather_attributes

['cloud_cover',
 'humidity',
 'pressure',
 'global_radiation',
 'precipitation',
 'sunshine',
 'temp_mean',
 'temp_min',
 'temp_max']

In [20]:
new_cols_to_keep = []
for col in cols_to_keep:
    if 'wind_speed' in col or 'snow_depth' in col:
        print(col)
        continue
    else:
        new_cols_to_keep.append(col)

BASEL_wind_speed
BASEL_snow_depth
DEBILT_wind_speed
DUSSELDORF_wind_speed
DUSSELDORF_snow_depth
HEATHROW_snow_depth
KASSEL_wind_speed
LJUBLJANA_wind_speed
MAASTRICHT_wind_speed
MADRID_wind_speed
MUNCHENB_snow_depth
OSLO_wind_speed
OSLO_snow_depth
SONNBLICK_wind_speed
VALENTIA_snow_depth


In [21]:
len(new_cols_to_keep)

132

In [22]:
station_dic = {station:0 for station in station_names}
for station in station_names:
    for col in cols_to_keep:
        if station in col:
            station_dic[station]+=1
            

In [23]:
station_dic

{'BASEL': 11,
 'BELGRADE': 9,
 'BUDAPEST': 9,
 'DEBILT': 10,
 'DUSSELDORF': 11,
 'HEATHROW': 10,
 'KASSEL': 9,
 'LJUBLJANA': 10,
 'MAASTRICHT': 10,
 'MADRID': 10,
 'MUNCHENB': 9,
 'OSLO': 11,
 'SONNBLICK': 10,
 'STOCKHOLM': 8,
 'VALENTIA': 10}

In [24]:
attribute_dic = {attribute:0 for attribute in weather_attributes}
for attribute in weather_attributes:
    for col in cols_to_keep:
        if attribute in col:
            attribute_dic[attribute]+=1
            

In [25]:
attribute_dic

{'cloud_cover': 14,
 'humidity': 14,
 'pressure': 14,
 'global_radiation': 15,
 'precipitation': 15,
 'sunshine': 15,
 'temp_mean': 15,
 'temp_min': 15,
 'temp_max': 15}

In [26]:
ljubljana_cols = [col for col in cols_to_keep if 'LJUBLJANA' in col]

In [27]:
ljubljana_cols

['LJUBLJANA_cloud_cover',
 'LJUBLJANA_wind_speed',
 'LJUBLJANA_humidity',
 'LJUBLJANA_pressure',
 'LJUBLJANA_global_radiation',
 'LJUBLJANA_precipitation',
 'LJUBLJANA_sunshine',
 'LJUBLJANA_temp_mean',
 'LJUBLJANA_temp_min',
 'LJUBLJANA_temp_max']

In [28]:
kassel_cols = [col for col in cols_to_keep if 'KASSEL' in col]

In [29]:
kassel_cols

['KASSEL_wind_speed',
 'KASSEL_humidity',
 'KASSEL_pressure',
 'KASSEL_global_radiation',
 'KASSEL_precipitation',
 'KASSEL_sunshine',
 'KASSEL_temp_mean',
 'KASSEL_temp_min',
 'KASSEL_temp_max']

In [30]:
weather_data = weather_data[new_cols_to_keep]

In [31]:
weather_data['KASSEL_cloud_cover']=weather_data['LJUBLJANA_cloud_cover']

In [32]:
munchen_cols = [col for col in cols_to_keep if 'MUNCHEN' in col]

In [33]:
munchen_cols

['MUNCHENB_cloud_cover',
 'MUNCHENB_humidity',
 'MUNCHENB_global_radiation',
 'MUNCHENB_precipitation',
 'MUNCHENB_snow_depth',
 'MUNCHENB_sunshine',
 'MUNCHENB_temp_mean',
 'MUNCHENB_temp_min',
 'MUNCHENB_temp_max']

In [34]:
weather_data['MUNCHENB_pressure']=weather_data['SONNBLICK_pressure']

In [35]:
stockholm_cols = [col for col in cols_to_keep if 'STOCKHOLM' in col]

In [36]:
stockholm_cols

['STOCKHOLM_cloud_cover',
 'STOCKHOLM_pressure',
 'STOCKHOLM_global_radiation',
 'STOCKHOLM_precipitation',
 'STOCKHOLM_sunshine',
 'STOCKHOLM_temp_mean',
 'STOCKHOLM_temp_min',
 'STOCKHOLM_temp_max']

In [37]:
weather_data['STOCKHOLM_humidity']=weather_data['OSLO_humidity']

In [38]:
weather_data.shape

(22950, 135)

In [39]:
save_path = r'C:\Users\kuohe\ClimateWins\02 Data\cleaned_weather_data.csv'
weather_data.to_csv(save_path, index=False)
print(f"Data has been successfully saved to {save_path}")

Data has been successfully saved to C:\Users\kuohe\ClimateWins\02 Data\cleaned_weather_data.csv


In [40]:
cleaned_weather_data = 'C:/Users/kuohe/ClimateWins/02 Data/cleaned_weather_data.csv'
cleaned_weather_data = pd.read_csv(cleaned_weather_data)

In [41]:
pleasant_weather_data_path = 'C:/Users/kuohe/ClimateWins/02 Data/pleasant weather.csv'
pleasant_weather_data = pd.read_csv(pleasant_weather_data_path)

In [42]:
pleasant_weather_data.drop(columns=['DATE'], inplace=True)

In [43]:
X = cleaned_weather_data.values.reshape(-1,15,9)
y = pleasant_weather_data

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

In [47]:
model = Sequential()
model.add(Conv1D(100, kernel_size=2, activation='relu', input_shape=(15,9)))
model.add(Dense(50, activation='relu'))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(15, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer = keras.optimizers.Adam(learning_rate=0.0000001), metrics=['accuracy'])
model.summary()

C:\Users\kuohe\anaconda2\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 14, 100)        │         1,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 14, 50)         │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 50)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 350)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │         5,265 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,215 (47.71 KB)

 Trainable params: 12,215 (47.71 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
model.fit(X_train,y_train,batch_size=16, epochs=100,verbose=1)
acc = model.evaluate(X_train,y_train)
print('Loss:', acc[0], 'Accuracy', acc[1])

Epoch 1/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0122 - loss: 14.0072
Epoch 2/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0132 - loss: 13.9515
Epoch 3/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0153 - loss: 13.7024
Epoch 4/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0145 - loss: 13.4858
Epoch 5/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0179 - loss: 13.5805
Epoch 6/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0164 - loss: 13.3835
Epoch 7/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.0206 - loss: 13.2885
Epoch 8/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0217 - loss: 13.3279
Epoch 9/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0234 - loss: 13.1798
Epoch 10/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0254 - loss: 13.0558
Epoch 11/100
1148/1148 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0275 - loss: 13.0500
Epoch 12

In [ ]:
# def create_rnn_model(input_shape, n_classes):
#     model = Sequential([
#         LSTM(units=300, return_sequences=False, input_shape=input_shape),
#         LSTM(units=200, return_sequences=False),
#         Dropout(0.2),
#         Dense(units=100, activation='sigmoid'),
#         Dropout(0.2),
#         Dense(units=n_classes, activation='softmax')
#     ])
#     model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#     return model

In [54]:
y_pred_binary = (y_pred > 0.5).astype(int)
print(classification_report(y_test, y_pred_binary))

              precision    recall  f1-score   support

           0       0.25      1.00      0.40      1134
           1       0.36      1.00      0.53      1619
           2       0.33      1.00      0.50      1495
           3       0.20      1.00      0.33       884
           4       0.22      1.00      0.36      1011
           5       0.21      1.00      0.35       985
           6       0.16      1.00      0.28       738
           7       0.28      1.00      0.44      1278
           8       0.21      1.00      0.35       970
           9       0.45      1.00      0.62      2056
          10       0.22      1.00      0.36       987
          11       0.15      1.00      0.27       708
          12       0.00      0.00      0.00         0
          13       0.17      1.00      0.29       761
          14       0.01      0.11      0.03       225

   micro avg       0.24      0.99      0.39     14851
   macro avg       0.21      0.87      0.34     14851
weighted avg       0.27   

C:\Users\kuohe\anaconda2\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\kuohe\anaconda2\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# EXERICSE 2.4 Part 2

In [7]:
import os
import time
import warnings
from math import floor
import operator

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import make_scorer, accuracy_score, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, BatchNormalization, Flatten, MaxPooling1D, LeakyReLU
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from scikeras.wrappers import KerasClassifier
from bayes_opt import BayesianOptimization

# Suppress warnings
warnings.filterwarnings('ignore')

# Display settings for pandas
pd.set_option("display.max_columns", None)


In [2]:
cleaned_weather_data_path = 'C:/Users/kuohe/ClimateWins/02 Data/cleaned_weather_data.csv'
pleasant_weather_data_path = 'C:/Users/kuohe/ClimateWins/02 Data/pleasant weather.csv'

In [3]:
cleaned_weather_data = pd.read_csv(cleaned_weather_data_path)
pleasant_weather_data = pd.read_csv(pleasant_weather_data_path)

In [4]:
X = cleaned_weather_data.values.reshape(-1, 15, 9)
y = pleasant_weather_data.drop('DATE', axis=1).values

In [5]:
y = np.argmax(y, axis=1)

print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (22950, 15, 9)
y shape: (22950,)


In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
# Ensure these variables are defined
# Define your training data and model parameters
X_train = np.random.rand(100, 10, 1)  # Example shape (number of samples, timesteps, input_dim)
y_train = np.random.randint(0, 2, 100)  # Example binary classification labels
timesteps = X_train.shape[1]
input_dim = X_train.shape[2]
n_classes = 2

# Custom scoring function
score_accuracy = make_scorer(accuracy_score)

def bay_area(neurons, activation, kernel, optimizer, learning_rate, batch_size, epochs,
              layers1, layers2, normalization, dropout, dropout_rate): 
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl', 'SGD']
    optimizerD= {'Adam':Adam(learning_rate=learning_rate), 'SGD':SGD(learning_rate=learning_rate),
                 'RMSprop':RMSprop(learning_rate=learning_rate), 'Adadelta':Adadelta(learning_rate=learning_rate),
                 'Adagrad':Adagrad(learning_rate=learning_rate), 'Adamax':Adamax(learning_rate=learning_rate),
                 'Nadam':Nadam(learning_rate=learning_rate), 'Ftrl':Ftrl(learning_rate=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
                   'elu', 'exponential', LeakyReLU, 'relu']
    
    neurons = round(neurons)
    kernel = round(kernel)
    activation = activationL[round(activation)]
    optimizer = optimizerD[optimizerL[round(optimizer)]]
    batch_size = round(batch_size)
    epochs = round(epochs)
    layers1 = round(layers1)
    layers2 = round(layers2)
    
    def cnn_model():
        model = Sequential()
        model.add(Conv1D(neurons, kernel_size=kernel, activation=activation, input_shape=(timesteps, input_dim)))
        if normalization > 0.5:
            model.add(BatchNormalization())
        for i in range(layers1):
            model.add(Dense(neurons, activation=activation))
        if dropout > 0.5:
            model.add(Dropout(dropout_rate, seed=123))
        for i in range(layers2):
            model.add(Dense(neurons, activation=activation))
        model.add(MaxPooling1D())
        model.add(Flatten())
        model.add(Dense(n_classes, activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
        return model
    
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=2, patience=20)
    nn = KerasClassifier(build_fn=cnn_model, epochs=epochs, batch_size=batch_size, verbose=2)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, y_train, scoring=score_accuracy, cv=kfold, fit_params={'callbacks': [es]}).mean()
    return score

# Define the parameter bounds for Bayesian Optimization
params = {
    'neurons': (10, 100),
    'activation': (0, 8),
    'kernel': (1, 5),
    'optimizer': (0, 7),
    'learning_rate': (0.0001, 0.1),
    'batch_size': (10, 100),
    'epochs': (10, 50),
    'layers1': (1, 5),
    'layers2': (1, 5),
    'normalization': (0, 1),
    'dropout': (0, 1),
    'dropout_rate': (0.1, 0.5)
}

start = time.time()
nn_opt = BayesianOptimization(f=bay_area, pbounds=params, random_state=42)
nn_opt.maximize(init_points=15, n_iter=4)
print('Search took %s minutes' % ((time.time() - start) / 60))


|   iter    |  target   | activa... | batch_... |  dropout  | dropou... |  epochs   |  kernel   |  layers1  |  layers2  | learni... |  neurons  | normal... | optimizer |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Epoch 1/16
1/1 - 1s - 1s/step - accuracy: 0.5750 - loss: 0.6951
Epoch 2/16
1/1 - 0s - 28ms/step - accuracy: 0.5875 - loss: 0.6910
Epoch 3/16
1/1 - 0s - 32ms/step - accuracy: 0.5875 - loss: 0.6887
Epoch 4/16
1/1 - 0s - 30ms/step - accuracy: 0.5875 - loss: 0.6869
Epoch 5/16
1/1 - 0s - 31ms/step - accuracy: 0.5875 - loss: 0.6854
Epoch 6/16
1/1 - 0s - 27ms/step - accuracy: 0.5875 - loss: 0.6842
Epoch 7/16
1/1 - 0s - 27ms/step - accuracy: 0.5875 - loss: 0.6831
Epoch 8/16
1/1 - 0s - 25ms/step - accuracy: 0.5875 - loss: 0.6823
Epoch 9/16
1/1 - 0s - 27ms/step - accuracy: 0.5875 - loss: 0.6816
Epoch 10/16
1/1 - 0s - 29ms/step - accuracy: 0.5875 - loss: 0.6810
E

Epoch 13/48
3/3 - 0s - 12ms/step - accuracy: 0.5500 - loss: 0.7083
Epoch 14/48
3/3 - 0s - 11ms/step - accuracy: 0.4125 - loss: 0.7156
Epoch 15/48
3/3 - 0s - 11ms/step - accuracy: 0.4875 - loss: 0.7816
Epoch 16/48
3/3 - 0s - 10ms/step - accuracy: 0.5500 - loss: 0.6986
Epoch 17/48
3/3 - 0s - 10ms/step - accuracy: 0.5875 - loss: 0.6839
Epoch 18/48
3/3 - 0s - 8ms/step - accuracy: 0.4500 - loss: 0.7691
Epoch 19/48
3/3 - 0s - 8ms/step - accuracy: 0.4625 - loss: 0.7967
Epoch 20/48
3/3 - 0s - 7ms/step - accuracy: 0.5625 - loss: 0.6840
Epoch 21/48
3/3 - 0s - 9ms/step - accuracy: 0.6125 - loss: 0.6747
Epoch 22/48
3/3 - 0s - 8ms/step - accuracy: 0.5875 - loss: 0.6992
Epoch 23/48
3/3 - 0s - 7ms/step - accuracy: 0.5000 - loss: 0.7193
Epoch 24/48
3/3 - 0s - 8ms/step - accuracy: 0.4875 - loss: 0.7180
Epoch 25/48
3/3 - 0s - 11ms/step - accuracy: 0.5000 - loss: 0.7045
Epoch 26/48
3/3 - 0s - 11ms/step - accuracy: 0.5875 - loss: 0.7026
Epoch 27/48
3/3 - 0s - 9ms/step - accuracy: 0.5250 - loss: 0.6991
Epo

Epoch 11/45
3/3 - 0s - 10ms/step - accuracy: 0.4125 - loss: 1.0192
Epoch 12/45
3/3 - 0s - 11ms/step - accuracy: 0.4125 - loss: 0.9513
Epoch 13/45
3/3 - 0s - 10ms/step - accuracy: 0.4125 - loss: 0.8587
Epoch 14/45
3/3 - 0s - 11ms/step - accuracy: 0.4125 - loss: 0.8176
Epoch 15/45
3/3 - 0s - 10ms/step - accuracy: 0.4125 - loss: 0.7838
Epoch 16/45
3/3 - 0s - 11ms/step - accuracy: 0.4125 - loss: 0.7756
Epoch 17/45
3/3 - 0s - 15ms/step - accuracy: 0.4125 - loss: 0.7638
Epoch 18/45
3/3 - 0s - 9ms/step - accuracy: 0.4500 - loss: 0.7287
Epoch 19/45
3/3 - 0s - 8ms/step - accuracy: 0.4750 - loss: 0.7113
Epoch 20/45
3/3 - 0s - 9ms/step - accuracy: 0.4625 - loss: 0.7092
Epoch 21/45
3/3 - 0s - 10ms/step - accuracy: 0.5125 - loss: 0.6929
Epoch 22/45
3/3 - 0s - 10ms/step - accuracy: 0.4875 - loss: 0.6944
Epoch 23/45
3/3 - 0s - 9ms/step - accuracy: 0.5500 - loss: 0.6893
Epoch 24/45
3/3 - 0s - 10ms/step - accuracy: 0.5750 - loss: 0.6761
Epoch 25/45
3/3 - 0s - 14ms/step - accuracy: 0.6000 - loss: 0.6813

Epoch 16/19
1/1 - 0s - 26ms/step - accuracy: 0.6000 - loss: 0.6472
Epoch 17/19
1/1 - 0s - 31ms/step - accuracy: 0.6500 - loss: 0.6468
Epoch 18/19
1/1 - 0s - 36ms/step - accuracy: 0.5875 - loss: 0.6497
Epoch 19/19
1/1 - 0s - 30ms/step - accuracy: 0.6375 - loss: 0.6436
1/1 - 0s - 81ms/step
Epoch 1/19
Epoch 1/19
Epoch 1/19
Epoch 1/19
| 11        | nan       | 6.46      | 90.65     | 0.318     | 0.144     | 19.12     | 2.708     | 4.272     | 4.443     | 0.0007945 | 55.97     | 0.4174    | 1.555     |
Epoch 1/31
2/2 - 2s - 809ms/step - accuracy: 0.5125 - loss: 6.5870
Epoch 2/31
2/2 - 0s - 18ms/step - accuracy: 0.5125 - loss: 3.3829
Epoch 3/31
2/2 - 0s - 11ms/step - accuracy: 0.4875 - loss: 1.3399
Epoch 4/31
2/2 - 0s - 18ms/step - accuracy: 0.4625 - loss: 1.1035
Epoch 5/31
2/2 - 0s - 13ms/step - accuracy: 0.4375 - loss: 0.7915
Epoch 6/31
2/2 - 0s - 13ms/step - accuracy: 0.5875 - loss: 0.6905
Epoch 7/31
2/2 - 0s - 14ms/step - accuracy: 0.5875 - loss: 0.6780
Epoch 8/31
2/2 - 0s - 12ms/step - 

Epoch 1/30
Epoch 1/30
Epoch 1/30
Epoch 1/30
| 15        | nan       | 0.3262    | 63.18     | 0.6776    | 0.1066    | 30.48     | 1.906     | 3.581     | 1.697     | 0.06912   | 44.81     | 0.9367    | 0.9626    |


ValueError: Input y contains NaN.

In [11]:
from tensorflow.keras.utils import to_categorical

# Assuming you have a test set y_test
y_test = np.random.randint(0, 2, 100)  # Example binary classification labels

# Convert y_test to one-hot format
y_test_one_hot = to_categorical(y_test)
print(y_test_one_hot)


[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]]


In [ ]:
# Use the best parameters from the optimization to train the final model
best_params = nn_opt.max['params']

model = bay_area(**best_params)

# Assuming you have a test set X_test and y_test
# Update y_test to one-hot format if needed
y_test_one_hot = np_utils.to_categorical(y_test)

# Make predictions and create a confusion matrix
y_pred = model.predict(X_test)
confusion_matrix = tf.math.confusion_matrix(y_test, y_pred)

# Plot the confusion matrix
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 7))
sns.heatmap(confusion_matrix, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()
